# Quantum Generative Adversarial Network (QGAN) Examples

This folder contains examples demonstrating the implementation of a Quantum Generative Adversarial Network (QGAN) using PyTorch and TorchQuantum.

## Contents

- `qgan_script.py`: Python script implementing the QGAN model.
- `qgan_notebook.ipynb`: Jupyter Notebook providing interactive examples of the QGAN model.

## Description

The QGAN model consists of a generator and a discriminator, both implemented as neural networks. The generator produces fake quantum data samples, while the discriminator tries to distinguish between real and fake samples. The two networks are trained adversarially, resulting in the generator learning to produce realistic quantum data.

## Usage

To run the examples:

1. Open `qgan_notebook.ipynb` in Jupyter Notebook.
2. Execute the notebook cells to see the QGAN model in action.
3. You can interactively change the number of qubits and latent dimensions to explore different configurations.

For more details, refer to the code and comments in the Python script and notebook.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchquantum as tq

class Generator(nn.Module):
    def __init__(self, n_qubits: int, latent_dim: int):
        super().__init__()
        self.n_qubits = n_qubits
        self.latent_dim = latent_dim

        # Quantum encoder
        self.encoder = tq.GeneralEncoder([
            {'input_idx': [i], 'func': 'rx', 'wires': [i]}
            for i in range(self.n_qubits)
        ])

        # RX gates
        self.rxs = nn.ModuleList([
            tq.RX(has_params=True, trainable=True) for _ in range(self.n_qubits)
        ])

    def forward(self, x):
        qdev = tq.QuantumDevice(n_wires=self.n_qubits, bsz=x.shape[0], device=x.device)
        self.encoder(qdev, x)

        for i in range(self.n_qubits):
            self.rxs[i](qdev, wires=i)

        return tq.measure(qdev)

class Discriminator(nn.Module):
    def __init__(self, n_qubits: int):
        super().__init__()
        self.n_qubits = n_qubits

        # Quantum encoder
        self.encoder = tq.GeneralEncoder([
            {'input_idx': [i], 'func': 'rx', 'wires': [i]}
            for i in range(self.n_qubits)
        ])

        # RX gates
        self.rxs = nn.ModuleList([
            tq.RX(has_params=True, trainable=True) for _ in range(self.n_qubits)
        ])

        # Quantum measurement
        self.measure = tq.MeasureAll(tq.PauliZ)

    def forward(self, x):
        qdev = tq.QuantumDevice(n_wires=self.n_qubits, bsz=x.shape[0], device=x.device)
        self.encoder(qdev, x)

        for i in range(self.n_qubits):
            self.rxs[i](qdev, wires=i)

        return self.measure(qdev)

class QGAN(nn.Module):
    def __init__(self, n_qubits: int, latent_dim: int):
        super().__init__()
        self.generator = Generator(n_qubits, latent_dim)
        self.discriminator = Discriminator(n_qubits)

    def forward(self, z):
        fake_data = self.generator(z)
        fake_output = self.discriminator(fake_data)
        return fake_output


In [ ]:
from ipywidgets import interact, IntSlider

@interact(n_qubits=IntSlider(min=1, max=10, step=1, value=4), latent_dim=IntSlider(min=1, max=10, step=1, value=2))
def create_model(n_qubits, latent_dim):
    model = QGAN(n_qubits, latent_dim)
    print(model)
